In [0]:
#1 Import the tools (from env import env, not used for this notebook)

from src import utils, excel

import openpyxl
import pandas as pd

from pyspark.sql import functions as F
from datetime import datetime
from openpyxl.styles import NamedStyle


In [0]:
#2 Load PIFU data (connecting to the file we cretaed in the qpat_load notebook)
from pyspark.sql import functions as F

# Load data
df_pifu_qa = spark.read.format('delta').load(
    'abfss://analytics-projects@udalstdataanalysisprod.dfs.core.windows.net/ElectiveRecovery/Projects/PIFU_Validation'
)


df_pifu_qa.display()

In [0]:
#3 This is making a data frame of the org names and regions that we can join to later to get the provider name
from pyspark.sql.types import StructType, StructField, StringType

data = [
    ("NV302", "Circle Bath Hospital", "South West"),
    ("NYG", "Sussex Community Dermatology Service", "South East"),
    ("R0A", "Manchester University NHS Foundation Trust", "North West"),
    ("R0B", "South Tyneside And Sunderland NHS Foundation Trust", "North East and Yorkshire"),
    ("R0D", "University Hospitals Dorset NHS Foundation Trust", "South West"),
    ("R1A", "Herefordshire And Worcestershire Health And Care NHS Trust", "Midlands"),
    ("R1D", "Shropshire Community Health NHS Trust", "Midlands"),
    ("R1F", "Isle of Wight NHS Trust", "South East"),
    ("R1H", "Barts Health NHS Trust", "London"),
    ("R1K", "London North West University Healthcare NHS Trust", "London"),
    ("R1L", "Essex Partnership University NHS Foundation Trust", "East of England"),
    ("RA2", "Royal Surrey County Hospital NHS Foundation Trust", "South East"),
    ("RA7", "University Hospitals Bristol And Weston NHS Foundation Trust", "South West"),
    ("RA9", "Torbay And South Devon NHS Foundation Trust", "South West"),
    ("RAE", "Bradford Teaching Hospitals NHS Foundation Trust", "North East and Yorkshire"),
    ("RAJ", "Mid And South Essex NHS Foundation Trust", "East of England"),
    ("RAL", "Royal Free London NHS Foundation Trust", "London"),
    ("RAN", "Royal National Orthopaedic Hospital NHS Trust", "London"),
    ("RAS", "The Hillingdon Hospitals NHS Foundation Trust", "London"),
    ("RAT", "North East London NHS Foundation Trust", "London"),
    ("RAX", "Kingston and Richmond NHS Foundation Trust", "London"),
    ("RBD", "Dorset County Hospital NHS Foundation Trust", "South West"),
    ("RBK", "Walsall Healthcare NHS Trust", "Midlands"),
    ("RBL", "Wirral University Teaching Hospital NHS Foundation Trust", "North West"),
    ("RBN", "Mersey and West Lancashire Teaching Hospitals NHS Trust", "North West"),
    ("RBQ", "Liverpool Heart And Chest Hospital NHS Foundation Trust", "North West"),
    ("RBS", "Alder Hey Children's NHS Foundation Trust", "North West"),
    ("RBT", "Mid Cheshire Hospitals NHS Foundation Trust", "North West"),
    ("RBV", "The Christie NHS Foundation Trust", "North West"),
    ("RC9", "Bedfordshire Hospitals NHS Foundation Trust", "East of England"),
    ("RCB", "York And Scarborough Teaching Hospitals NHS Foundation Trust", "North East and Yorkshire"),
    ("RCD", "Harrogate And District NHS Foundation Trust", "North East and Yorkshire"),
    ("RCF", "Airedale NHS Foundation Trust", "North East and Yorkshire"),
    ("RCU", "Sheffield Children's NHS Foundation Trust", "North East and Yorkshire"),
    ("RCX", "The Queen Elizabeth Hospital, King's Lynn, NHS Foundation Trust", "East of England"),
    ("RD1", "Royal United Hospitals Bath NHS Foundation Trust", "South West"),
    ("RD8", "Milton Keynes University Hospital NHS Foundation Trust", "East of England"),
    ("RDE", "East Suffolk And North Essex NHS Foundation Trust", "East of England"),
    ("RDR", "Sussex Community NHS Foundation Trust", "South East"),
    ("RDU", "Frimley Health NHS Foundation Trust", "South East"),
    ("RDY", "Dorset Healthcare University NHS Foundation Trust", "South West"),
    ("REF", "Royal Cornwall Hospitals NHS Trust", "South West"),
    ("REM", "Liverpool University Hospitals NHS Foundation Trust", "North West"),
    ("REN", "The Clatterbridge Cancer Centre NHS Foundation Trust", "North West"),
    ("REP", "Liverpool Women's NHS Foundation Trust", "North West"),
    ("RET", "The Walton Centre NHS Foundation Trust", "North West"),
    ("RF4", "Barking, Havering And Redbridge University Hospitals NHS Trust", "London"),
    ("RFF", "Barnsley Hospital NHS Foundation Trust", "North East and Yorkshire"),
    ("RFR", "The Rotherham NHS Foundation Trust", "North East and Yorkshire"),
    ("RFS", "Chesterfield Royal Hospital NHS Foundation Trust", "Midlands"),
    ("RGM", "Royal Papworth Hospital NHS Foundation Trust", "East of England"),
    ("RGN", "North West Anglia NHS Foundation Trust", "East of England"),
    ("RGP", "James Paget University Hospitals NHS Foundation Trust", "East of England"),
    ("RGR", "West Suffolk NHS Foundation Trust", "East of England"),
    ("RGT", "Cambridge University Hospitals NHS Foundation Trust", "East of England"),
    ("RH5", "Somerset NHS Foundation Trust", "South West"),
    ("RH8", "Royal Devon University Healthcare NHS Foundation Trust", "South West"),
    ("RHA", "Nottinghamshire Healthcare NHS Foundation Trust", "Midlands"),
    ("RHM", "University Hospital Southampton NHS Foundation Trust", "South East"),
    ("RHQ", "Sheffield Teaching Hospitals NHS Foundation Trust", "North East and Yorkshire"),
    ("RHU", "Portsmouth Hospitals University National Health Service Trust", "South East"),
    ("RHW", "Royal Berkshire NHS Foundation Trust", "South East"),
    ("RJ1", "Guy's And St Thomas' NHS Foundation Trust", "London"),
    ("RJ2", "Lewisham And Greenwich NHS Trust", "London"),
    ("RJ6", "Croydon Health Services NHS Trust", "London"),
    ("RJ7", "St George's University Hospitals NHS Foundation Trust", "London"),
    ("RJ8", "Cornwall Partnership NHS Foundation Trust", "South West"),
    ("RJC", "South Warwickshire NHS Foundation Trust", "Midlands"),
    ("RJE", "University Hospitals of North Midlands NHS Trust", "Midlands"),
    ("RJL", "Northern Lincolnshire And Goole NHS Foundation Trust", "North East and Yorkshire"),
    ("RJN", "East Cheshire NHS Trust", "North West"),
    ("RJR", "Countess of Chester Hospital NHS Foundation Trust", "North West"),
    ("RJZ", "King's College Hospital NHS Foundation Trust", "London"),
    ("RK5", "Sherwood Forest Hospitals NHS Foundation Trust", "Midlands"),
    ("RK9", "University Hospitals Plymouth NHS Trust", "South West"),
    ("RKB", "University Hospitals Coventry And Warwickshire NHS Trust", "Midlands"),
    ("RKE", "Whittington Health NHS Trust", "London"),
    ("RKL", "West London NHS Trust", "London"),
    ("RL1", "The Robert Jones And Agnes Hunt Orthopaedic Hospital NHS Foundation Trust", "Midlands"),
    ("RL4", "The Royal Wolverhampton NHS Trust", "Midlands"),
    ("RLQ", "Wye Valley NHS Trust", "Midlands"),
    ("RLT", "George Eliot Hospital NHS Trust", "Midlands"),
    ("RLY", "North Staffordshire Combined Healthcare NHS Trust", "Midlands"),
    ("RM1", "Norfolk And Norwich University Hospitals NHS Foundation Trust", "East of England"),
    ("RM3", "Northern Care Alliance NHS Foundation Trust", "North West"),
    ("RMC", "Bolton NHS Foundation Trust", "North West"),
    ("RMP", "Tameside And Glossop Integrated Care NHS Foundation Trust", "North West"),
    ("RMY", "Norfolk And Suffolk NHS Foundation Trust", "East of England"),
    ("RN3", "Great Western Hospitals NHS Foundation Trust", "South West"),
    ("RN5", "Hampshire Hospitals NHS Foundation Trust", "South East"),
    ("RN7", "Dartford And Gravesham NHS Trust", "South East"),
    ("RNA", "The Dudley Group NHS Foundation Trust", "Midlands"),
    ("RNK", "Tavistock And Portman NHS Foundation Trust", "London"),
    ("RNN", "North Cumbria Integrated Care NHS Foundation Trust", "North East and Yorkshire"),
    ("RNQ", "Kettering General Hospital NHS Foundation Trust", "Midlands"),
    ("RNS", "Northampton General Hospital NHS Trust", "Midlands"),
    ("RNU", "Oxford Health NHS Foundation Trust", "South East"),
    ("RNZ", "Salisbury NHS Foundation Trust", "South West"),
    ("RP1", "Northamptonshire Healthcare NHS Foundation Trust", "Midlands"),
    ("RP4", "Great Ormond Street Hospital For Children NHS Foundation Trust", "London"),
    ("RP5", "Doncaster And Bassetlaw Teaching Hospitals NHS Foundation Trust", "North East and Yorkshire"),
    ("RP6", "Moorfields Eye Hospital NHS Foundation Trust", "London"),
    ("RP7", "Lincolnshire Partnership NHS Foundation Trust", "Midlands"),
    ("RPA", "Medway NHS Foundation Trust", "South East"),
    ("RPC", "Queen Victoria Hospital NHS Foundation Trust", "South East"),
    ("RPG", "Oxleas NHS Foundation Trust", "London"),
    ("RPY", "The Royal Marsden NHS Foundation Trust", "London"),
    ("RQ3", "Birmingham Women's And Children's NHS Foundation Trust", "Midlands"),
    ("RQM", "Chelsea And Westminster Hospital NHS Foundation Trust", "London"),
    ("RQW", "The Princess Alexandra Hospital NHS Trust", "East of England"),
    ("RQX", "Homerton University Hospital NHS Foundation Trust", "London"),
    ("RQY", "South West London And St George's Mental Health NHS Trust", "London"),
    ("RR7", "Gateshead Health NHS Foundation Trust", "North East and Yorkshire"),
    ("RR8", "Leeds Teaching Hospitals NHS Trust", "North East and Yorkshire"),
    ("RRE", "Midlands Partnership NHS Foundation Trust", "Midlands"),
    ("RRF", "Wrightington, Wigan And Leigh NHS Foundation Trust", "North West"),
    ("RRJ", "The Royal Orthopaedic Hospital NHS Foundation Trust", "Midlands"),
    ("RRK", "University Hospitals Birmingham NHS Foundation Trust", "Midlands"),
    ("RRP", "Barnet, Enfield And Haringey Mental Health NHS Trust", "London"),
    ("RRV", "University College London Hospitals NHS Foundation Trust", "London"),
    ("RT1", "Cambridgeshire And Peterborough NHS Foundation Trust", "East of England"),
    ("RT2", "Pennine Care NHS Foundation Trust", "North West"),
    ("RT5", "Leicestershire Partnership NHS Trust", "Midlands"),
    ("RTD", "The Newcastle Upon Tyne Hospitals NHS Foundation Trust", "North East and Yorkshire"),
    ("RTE", "Gloucestershire Hospitals NHS Foundation Trust", "South West"),
    ("RTF", "Northumbria Healthcare NHS Foundation Trust", "North East and Yorkshire"),
    ("RTG", "University Hospitals of Derby And Burton NHS Foundation Trust", "Midlands"),
    ("RTH", "Oxford University Hospitals NHS Foundation Trust", "South East"),
    ("RTK", "Ashford And St Peter's Hospitals NHS Foundation Trust", "South East"),
    ("RTP", "Surrey And Sussex Healthcare NHS Trust", "South East"),
    ("RTQ", "Gloucestershire Health And Care NHS Foundation Trust", "South West"),
    ("RTR", "South Tees Hospitals NHS Foundation Trust", "North East and Yorkshire"),
    ("RTX", "University Hospitals of Morecambe Bay NHS Foundation Trust", "North West"),
    ("RV3", "Central And North West London NHS Foundation Trust", "London"),
    ("RV5", "South London And Maudsley NHS Foundation Trust", "London"),
    ("RV9", "Humber Teaching NHS Foundation Trust", "North East and Yorkshire"),
    ("RVJ", "North Bristol NHS Trust", "South West"),
    ("RVN", "Avon And Wiltshire Mental Health Partnership NHS Trust", "South West"),
    ("RVR", "Epsom And St Helier University Hospitals NHS Trust", "London"),
    ("RVV", "East Kent Hospitals University NHS Foundation Trust", "South East"),
    ("RVW", "North Tees And Hartlepool NHS Foundation Trust", "North East and Yorkshire"),
    ("RW1", "Hampshire and Isle of Wight Healthcare NHS Foundation Trust", "South East"),
    ("RW4", "Mersey Care NHS Foundation Trust", "North West"),
    ("RW5", "Lancashire & South Cumbria NHS Foundation Trust", "North West"),
    ("RWA", "Hull University Teaching Hospitals NHS Trust", "North East and Yorkshire"),
    ("RWD", "United Lincolnshire Hospitals NHS Trust", "Midlands"),
    ("RWE", "University Hospitals of Leicester NHS Trust", "Midlands"),
    ("RWF", "Maidstone And Tunbridge Wells NHS Trust", "South East"),
    ("RWG", "West Hertfordshire Teaching Hospitals NHS Trust", "East of England"),
    ("RWH", "East And North Hertfordshire NHS Trust", "East of England"),
    ("RWJ", "Stockport NHS Foundation Trust", "North West"),
    ("RWK", "East London NHS Foundation Trust", "London"),
    ("RWP", "Worcestershire Acute Hospitals NHS Trust", "Midlands"),
    ("RWR", "Hertfordshire Partnership University NHS Foundation Trust", "East of England"),
    ("RWV", "Devon Partnership NHS Trust", "South West"),
    ("RWW", "Warrington And Halton Teaching Hospitals NHS Foundation Trust", "North West"),
    ("RWX", "Berkshire Healthcare NHS Foundation Trust", "South East"),
    ("RWY", "Calderdale And Huddersfield NHS Foundation Trust", "North East and Yorkshire"),
    ("RX1", "Nottingham University Hospitals NHS Trust", "Midlands"),
    ("RX2", "Sussex Partnership NHS Foundation Trust", "South East"),
    ("RX4", "Cumbria, Northumberland, Tyne And Wear NHS Foundation Trust", "North East and Yorkshire"),
    ("RXA", "Cheshire And Wirral Partnership NHS Foundation Trust", "North West"),
    ("RXC", "East Sussex Healthcare NHS Trust", "South East"),
    ("RXE", "Rotherham Doncaster And South Humber NHS Foundation Trust", "North East and Yorkshire"),
    ("RXF", "Mid Yorkshire Hospitals NHS Trust", "North East and Yorkshire"),
    ("RXG", "South West Yorkshire Partnership NHS Foundation Trust", "North East and Yorkshire"),
    ("RXK", "Sandwell And West Birmingham Hospitals NHS Trust", "Midlands"),
    ("RXL", "Blackpool Teaching Hospitals NHS Foundation Trust", "North West"),
    ("RXM", "Derbyshire Healthcare NHS Foundation Trust", "Midlands"),
    ("RXN", "Lancashire Teaching Hospitals NHS Foundation Trust", "North West"),
    ("RXP", "County Durham And Darlington NHS Foundation Trust", "North East and Yorkshire"),
    ("RXQ", "Buckinghamshire Healthcare NHS Trust", "South East"),
    ("RXR", "East Lancashire Hospitals NHS Trust", "North West"),
    ("RXT", "Birmingham And Solihull Mental Health NHS Foundation Trust", "Midlands"),
    ("RXW", "The Shrewsbury And Telford Hospital NHS Trust", "Midlands"),
    ("RXX", "Surrey And Borders Partnership NHS Foundation Trust", "South East"),
    ("RXY", "Kent And Medway NHS And Social Care Partnership Trust", "South East"),
    ("RY2", "Bridgewater Community Healthcare NHS Foundation Trust", "North West"),
    ("RY3", "Norfolk Community Health And Care NHS Trust", "East of England"),
    ("RY4", "Hertfordshire Community NHS Trust", "East of England"),
    ("RY5", "Lincolnshire Community Health Services NHS Trust", "Midlands"),
    ("RY6", "Leeds Community Healthcare NHS Trust", "North East and Yorkshire"),
    ("RY7", "Wirral Community Health And Care NHS Foundation Trust", "North West"),
    ("RY8", "Derbyshire Community Health Services NHS Foundation Trust", "Midlands"),
    ("RYG", "Coventry And Warwickshire Partnership NHS Trust", "Midlands"),
    ("RYJ", "Imperial College Healthcare NHS Trust", "London"),
    ("RYR", "University Hospitals Sussex NHS Foundation Trust", "South East"),
    ("RYV", "Cambridgeshire Community Services NHS Trust", "East of England"),
    ("RYW", "Birmingham Community Healthcare NHS Foundation Trust", "Midlands"),
    ("RYX", "Central London Community Healthcare NHS Trust", "London"),
    ("RYY", "Kent Community Health NHS Foundation Trust", "South East"),
    ("TAD", "Bradford District Care NHS Foundation Trust", "North East and Yorkshire")
]

schema = StructType([
    StructField("Org Code", StringType(), True),
    StructField("Org Name", StringType(), True),
    StructField("Region Name", StringType(), True)
])

org_name = spark.createDataFrame(data, schema)

org_name.display()

In [0]:
#4 Bringing back moved and discharged columns and value from raw df & creating a join to get OrgName
from pyspark.sql import functions as F

# Step 1: Filter and relabel the metrics
df_moved_discharged = (
    df_pifu_qa
    .where(F.col("EROC_DerMetric").isin("PIFUSRTMV01", "PIFUSRTDC01"))
    .where(F.col("EROC_DerMonth") > '2023-03-01')
    .where(F.col("EROC_Latest_Flag") == 1)
    .withColumn("EROC_DerMetric", F.lit("Moved&Discharged"))  # Rename both to single value
    .groupby(
        "EROC_DerMonth",
        "EROC_DerProviderCode",
        "EROC_DerTFC",
        "RegionName",
        "EROC_DerMetric"
    )
    .agg(F.sum("EROC_Value").alias("Value"))
    .orderBy(
        "EROC_DerMonth",
        "EROC_DerProviderCode"
    )
)

# Step 2: Join to get OrgName
df_moved_discharged = df_moved_discharged.join(
    org_name.withColumnRenamed("Org Name", "OrgName"),
    df_moved_discharged["EROC_DerProviderCode"] == org_name["Org Code"],
    "left"
).drop(org_name["Org Code"])

# Display result
df_moved_discharged.display()

In [0]:
#5 making the pivot of (df_moved_discharged)
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")#create the pivoted data frame

df_pivoted = (
    df_moved_discharged
    .groupBy("EROC_DerMonth", "RegionName", "EROC_DerProviderCode", "OrgName")
    .pivot("EROC_DerMetric")  # creates columns for each metric
    .agg(F.first("Value"))    # or F.sum if there are multiple rows
    .na.fill(0)  # replace nulls with 0
    .orderBy("EROC_DerMonth", "RegionName", "EROC_DerProviderCode")
)

df_pivoted.display()


In [0]:
#6 each data refresh needs to have the old widgest removed
dbutils.widgets.removeAll()

In [0]:
#7 Creating the data structure for the chart 
from pyspark.sql.functions import to_date, col
from datetime import datetime
import plotly.express as px

# Step 1: Convert date string to proper date type
df_moved_discharged = df_moved_discharged.withColumn(
    "EROC_DerMonth", to_date(col("EROC_DerMonth"), "yyyy-MM-dd")
)

# Step 2: Register the DataFrame as a temp SQL view
df_moved_discharged.createOrReplaceTempView("moved_discharged")

# Step 3: Get base DataFrame with valid rows
df_line_chart = spark.sql("""
    SELECT
        EROC_DerMonth,
        RegionName,
        EROC_DerProviderCode,
        OrgName,
        EROC_DerMetric,
        Value
    FROM
        moved_discharged
    WHERE
        EROC_DerMonth IS NOT NULL
        AND RegionName IS NOT NULL
        AND EROC_DerProviderCode IS NOT NULL
        AND OrgName IS NOT NULL
        AND EROC_DerMetric IS NOT NULL
""")

# Step 4: Fetch distinct values and date range
valid_regions = sorted(set(str(r[0]) for r in df_line_chart.select("RegionName").distinct().collect()))
valid_providers = sorted(set(str(p[0]) for p in df_line_chart.select("EROC_DerProviderCode").distinct().collect()))
valid_metrics = sorted(set(str(m[0]) for m in df_line_chart.select("EROC_DerMetric").distinct().collect()))
min_date, max_date = df_line_chart.selectExpr("min(EROC_DerMonth)", "max(EROC_DerMonth)").first()


In [0]:
#8 Creating the filters in numerical order

# 0. Region dropdown
valid_regions = sorted(set(str(p[0]) for p in df_line_chart.select("RegionName").distinct().collect()))
dbutils.widgets.dropdown("1_Region", "All", ["All"] + valid_regions)

# Get the selected region
selected_region = dbutils.widgets.get("1_Region")

# 1. Filtered provider dropdown based on selected region
if selected_region == "All":
    filtered_df = df_line_chart
else:
    filtered_df = df_line_chart.filter(df_line_chart["RegionName"] == selected_region)

valid_providers = sorted(set(str(p[0]) for p in filtered_df.select("EROC_DerProviderCode").distinct().collect()))
dbutils.widgets.dropdown("2_Provider", "All", ["All"] + valid_providers)

# 2. Metric dropdown
valid_metrics = sorted(set(str(m[0]) for m in df_line_chart.select("EROC_DerMetric").distinct().collect()))
dbutils.widgets.dropdown("3_Metric", "All", ["All"] + valid_metrics)

# 3. Start date text box
min_date = df_line_chart.selectExpr("min(EROC_DerMonth)").first()[0]
dbutils.widgets.text("4_StartDate", min_date.strftime("%Y-%m-%d") if min_date else "")

# 4. End date text box
max_date = df_line_chart.selectExpr("max(EROC_DerMonth)").first()[0]
dbutils.widgets.text("5_EndDate", max_date.strftime("%Y-%m-%d") if max_date else "")


In [0]:
#9 Creating filter settings and making filter 2 dynamic to filter 1
selected_region = dbutils.widgets.get("1_Region")
selected_provider = dbutils.widgets.get("2_Provider")
selected_metric = dbutils.widgets.get("3_Metric")
start_date_str = dbutils.widgets.get("4_StartDate")
end_date_str = dbutils.widgets.get("5_EndDate")

# Step 7: Validate and parse dates
def safe_parse(date_str, fallback):
    try:
        return datetime.strptime(date_str, "%Y-%m-%d").date()
    except:
        return fallback

start_date = safe_parse(start_date_str, min_date)
end_date = safe_parse(end_date_str, max_date)

# Step 8: Apply filtering
df_filtered = df_line_chart.filter((col("EROC_DerMonth") >= start_date) & (col("EROC_DerMonth") <= end_date))

# Filter by region (if not "All")
if selected_region != "All":
    df_filtered = df_filtered.filter(col("RegionName") == selected_region)

# Filter by provider (if not "All")
if selected_provider != "All" and selected_provider in valid_providers:
    df_filtered = df_filtered.filter(col("EROC_DerProviderCode") == selected_provider)

# Filter by metric (if not "All")
if selected_metric != "All" and selected_metric in valid_metrics:
    df_filtered = df_filtered.filter(col("EROC_DerMetric") == selected_metric)

# Step 9: Pivot and display
# To see EROC_DerProviderCode in the visualization, do not pivot away this column
display(df_filtered.select("EROC_DerMonth", "EROC_DerProviderCode", "EROC_DerMetric", "Value"))

# Step 10: Optional Plotly multi-line chart for selected metric
if selected_metric != "All":
    # Aggregate per provider code per month
    pdf = (
        df_filtered
        .groupBy("EROC_DerMonth", "EROC_DerProviderCode")
        .agg(F.sum("Value").alias("Value"))
        .orderBy("EROC_DerMonth")
        .toPandas()
    )

    fig1 = px.line(
        pdf,
        x="EROC_DerMonth",
        y="Value",
        color="EROC_DerProviderCode",  # One line per provider code
        title=f"{selected_metric} Trend by Provider Code in {selected_region}" if selected_region != "All" else f"{selected_metric} Trend by Provider Code (All Regions)"
    )
    fig1.show()

Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql import functions as F

# Get filter values from widgets
selected_region = dbutils.widgets.get("1_Region")
selected_provider = dbutils.widgets.get("2_Provider")
#selected_metric = dbutils.widgets.get("3_Metric")
start_date_str = dbutils.widgets.get("4_StartDate")
end_date_str = dbutils.widgets.get("5_EndDate")

def safe_parse(date_str, fallback):
    try:
        return datetime.strptime(date_str, "%Y-%m-%d").date()
    except:
        return fallback

min_date = df_pifu_qa.selectExpr("min(EROC_DerMonth)").first()[0]
max_date = df_pifu_qa.selectExpr("max(EROC_DerMonth)").first()[0]
start_date = safe_parse(start_date_str, min_date)
end_date = safe_parse(end_date_str, max_date)

# Filter by region if not "All"
df_filtered = df_pifu_qa.where(F.col("EROC_DerMetric") == "PIFUTOTAL01") \
    .where(F.col("EROC_Latest_Flag") == 1) \
    .where(F.col("EROC_DerMonth") >= start_date) \
    .where(F.col("EROC_DerMonth") <= end_date)

if selected_region != "All":
    # Get org codes for selected region
    region_org_codes = [r["Org Code"] for r in org_name.filter(F.col("Region Name") == selected_region).select("Org Code").collect()]
    df_filtered = df_filtered.where(F.col("EROC_DerProviderCode").isin(region_org_codes))

if selected_provider != "All":
    df_filtered = df_filtered.where(F.col("EROC_DerProviderCode") == selected_provider)

df_PIFUTOTAL01 = (
    df_filtered
    .groupby(
        "EROC_DerMonth",
        "EROC_DerProviderCode",
        "EROC_DerMetric"
    )
    .agg(F.sum("EROC_Value").alias("Value"))
    .orderBy(
        "EROC_DerMonth",
        "EROC_DerProviderCode"
    )
)

display(df_PIFUTOTAL01)

Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql import functions as F
from datetime import datetime

# Get end_date_str from widget and parse it
end_date_str = dbutils.widgets.get("5_EndDate")
def safe_parse(date_str, fallback):
    try:
        return datetime.strptime(date_str, "%Y-%m-%d").date()
    except:
        return fallback

min_date = df_pifu_qa.selectExpr("min(EROC_DerMonth)").first()[0]
max_date = df_pifu_qa.selectExpr("max(EROC_DerMonth)").first()[0]
end_date = safe_parse(end_date_str, max_date)

df_blank_null = (
    df_pifu_qa
    .where(F.col("EROC_DerMetric").isin(
        "PIFUSRTMV01", "PIFUSRTDC01", "PIFUTOTAL01", "PIFUCOMP001", "PIFUBOOK001", "PIFUDNA0001"
    ))
    .where(F.col("EROC_DerMonth") == end_date)
    .where(F.col("EROC_Latest_Flag") == 1)
    .where(~F.col("Submission_type").rlike("^01\| Complete return:|^02\| Partial return:"))
)

df_blank_null = df_blank_null.join(
    org_name.withColumnRenamed("Org Name", "OrgName"),
    df_blank_null["EROC_DerProviderCode"] == org_name["Org Code"],
    "left"
).drop(org_name["Org Code"])

df_blank_null = df_blank_null.groupBy(
    "EROC_DerMonth",
    "EROC_DerProviderCode",
    "OrgName",
    "Submission_type",  
    "RegionName",
).agg(F.sum("EROC_Value").alias("Value")).orderBy(
    "EROC_DerMonth",
    "RegionName",
    "EROC_DerProviderCode"
)

display(df_blank_null)

In [0]:
from pyspark.sql import functions as F
from datetime import datetime

selected_region = dbutils.widgets.get("1_Region")
selected_provider = dbutils.widgets.get("2_Provider")
selected_metric = dbutils.widgets.get("3_Metric")
start_date_str = dbutils.widgets.get("4_StartDate")
end_date_str = dbutils.widgets.get("5_EndDate")

def safe_parse(date_str, fallback):
    try:
        return datetime.strptime(date_str, "%Y-%m-%d").date()
    except:
        return fallback

min_date = df_pifu_qa.selectExpr("min(EROC_DerMonth)").first()[0]
max_date = df_pifu_qa.selectExpr("max(EROC_DerMonth)").first()[0]
start_date = safe_parse(start_date_str, min_date)
end_date = safe_parse(end_date_str, max_date)

def get_filtered_df(latest_flag):
    df = (
        df_pifu_qa
        .filter(F.col("EROC_Latest_Flag") == latest_flag)
        .filter((F.col("EROC_DerMonth") >= F.lit(start_date)) & (F.col("EROC_DerMonth") <= F.lit(end_date)))
    )

    if selected_provider != "All":
        df = df.filter(F.col("EROC_DerProviderCode") == selected_provider)

    if selected_metric != "All":
        df = df.filter(F.col("EROC_DerMetric") == selected_metric)

    df_grouped = (
        df.groupBy("EROC_DerMonth", "EROC_DerMetric")
          .agg(F.sum("EROC_Value").alias("Value"))
          .orderBy("EROC_DerMonth", "EROC_DerMetric")
    )
    
    return df_grouped

df_latest_1 = get_filtered_df(1)
df_latest_0 = get_filtered_df(0)

display(df_latest_1)  # Stacked bar: x = EROC_DerMonth, series = EROC_DerMetric, y = Value
display(df_latest_0)  # Stacked bar: x = EROC_DerMonth, series = EROC_DerMetric, y = Value

In [0]:
# The combined chart with dynamic provider filtering based on selected region
from pyspark.sql import functions as F
from datetime import datetime

selected_region = dbutils.widgets.get("1_Region")
selected_provider = dbutils.widgets.get("2_Provider")
selected_metric = dbutils.widgets.get("3_Metric")
start_date_str = dbutils.widgets.get("4_StartDate")
end_date_str = dbutils.widgets.get("5_EndDate")

def safe_parse(date_str, fallback):
    try:
        return datetime.strptime(date_str, "%Y-%m-%d").date()
    except:
        return fallback

min_date = df_pifu_qa.selectExpr("min(EROC_DerMonth)").first()[0]
max_date = df_pifu_qa.selectExpr("max(EROC_DerMonth)").first()[0]
start_date = safe_parse(start_date_str, min_date)
end_date = safe_parse(end_date_str, max_date)

# Filter org_name by selected_region to get relevant Org Codes
if selected_region != "All":
    org_codes_in_region = [
        r["Org Code"] for r in org_name.filter(F.col("Region Name") == selected_region).select("Org Code").collect()
    ]
else:
    org_codes_in_region = [r["Org Code"] for r in org_name.select("Org Code").collect()]

def get_filtered_df_region(latest_flag):
    df = (
        df_pifu_qa
        .filter(F.col("EROC_Latest_Flag") == latest_flag)
        .filter((F.col("EROC_DerMonth") >= F.lit(start_date)) & (F.col("EROC_DerMonth") <= F.lit(end_date)))
    )
    if selected_region != "All":
        df = df.filter(F.col("EROC_DerProviderCode").isin(org_codes_in_region))
    if selected_provider != "All":
        df = df.filter(F.col("EROC_DerProviderCode") == selected_provider)
    if selected_metric != "All":
        df = df.filter(F.col("EROC_DerMetric") == selected_metric)
    df_grouped = (
        df.groupBy("EROC_DerMonth", "EROC_DerMetric")
          .agg(F.sum("EROC_Value").alias("Value"))
          .orderBy("EROC_DerMonth", "EROC_DerMetric")
    )
    return df_grouped

df_latest_1 = get_filtered_df_region(1).withColumn("Latest_Flag", F.lit(1))
df_latest_0 = get_filtered_df_region(0).withColumn("Latest_Flag", F.lit(0))

df_combined = df_latest_1.unionByName(df_latest_0)

display(df_combined)

Databricks visualization. Run in Databricks to view.

In [0]:
# Splitting the chart
selected_region = dbutils.widgets.get("1_Region")
selected_provider = dbutils.widgets.get("2_Provider")
selected_metric = dbutils.widgets.get("3_Metric")
start_date_str = dbutils.widgets.get("4_StartDate")
end_date_str = dbutils.widgets.get("5_EndDate")

df_latest_1 = get_filtered_df(1).withColumn("Latest_Flag", F.lit(1))
df_latest_0 = get_filtered_df(0).withColumn("Latest_Flag", F.lit(0))

display(df_latest_0)  # Stacked bar: x = EROC_DerMonth, series = EROC_DerMetric, y = Value, filter Latest_Flag = 0
display(df_latest_1)  # Stacked bar: x = EROC_DerMonth, series = EROC_DerMetric, y = Value, filter Latest_Flag = 1

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql import functions as F
from datetime import datetime
from dateutil.relativedelta import relativedelta

# 1. Filter latest flagged data and cast
validated = (
    df_pifu_qa
    .filter(F.col("EROC_Latest_Flag") == 1)
    .select(
        "EROC_DerProviderCode",
        F.col("SubmittedDatetime").cast("date").alias("SubmittedDate"),
        F.col("EROC_DerMonth").cast("date").alias("EROC_DerMonth"),
        "EROC_Value"
    )
)

# 2. Find latest SubmittedDate max
latest_submitted_date = validated.agg(F.max("SubmittedDate").alias("max_submitted_date")).collect()[0]["max_submitted_date"]

# 3. Calculate previous month from latest SubmittedDate
prev_month_date = (latest_submitted_date - relativedelta(months=1))
latest_year, latest_month = latest_submitted_date.year, latest_submitted_date.month
prev_year, prev_month = prev_month_date.year, prev_month_date.month

# 4. Providers who submitted in latest month
providers_latest_submission = (
    validated
    .filter(
        (F.year("SubmittedDate") == latest_year) &
        (F.month("SubmittedDate") == latest_month)
    )
    .select("EROC_DerProviderCode")
    .distinct()
)

# 5. Providers who have data in previous month (EROC_DerMonth = previous month)
providers_with_prev_month_data = (
    validated
    .filter(
        (F.year("EROC_DerMonth") == prev_year) &
        (F.month("EROC_DerMonth") == prev_month)
    )
    .select("EROC_DerProviderCode")
    .distinct()
)

# 6. Flagged providers = submitted in latest month BUT no data for previous month
flagged_providers = (
    providers_latest_submission.alias("pls")
    .join(
        providers_with_prev_month_data.alias("pwp"),
        on="EROC_DerProviderCode",
        how="left_anti"
    )
)

# 7. For flagged providers, get the detailed rows for latest SubmittedDate month
result = (
    flagged_providers.alias("fp")
    .join(
        validated.alias("v"),
        on="EROC_DerProviderCode"
    )
    .filter(
        (F.year("v.SubmittedDate") == latest_year) &
        (F.month("v.SubmittedDate") == latest_month)
    )
    .groupBy("fp.EROC_DerProviderCode", "v.SubmittedDate", "v.EROC_DerMonth")
    .agg(F.sum("v.EROC_Value").alias("TotalValue"))
    .orderBy("fp.EROC_DerProviderCode")
)

display(result)

In [0]:
from pyspark.sql import functions as F

# 1 Get the latest month (truncated to month) in the data
latest_month = (
    df_pifu_qa
    .select(F.max(F.trunc("SubmittedDatetime", "month")).alias("latest_month"))
    .collect()[0]["latest_month"]
)

# 2 Filter rows matching conditions
filtered_df = (
    df_pifu_qa
    .filter(F.col("EROC_Latest_Flag") == 1)
    .filter(F.col("EROC_Value") > 0)
    .filter(F.trunc("SubmittedDatetime", "month") == F.lit(latest_month))
)

# 3 Count distinct Organisation Codes and create a DataFrame for dashboard
result_df = (
    filtered_df
    .select("EROC_DerProviderCode")
    .distinct()
    .agg(F.count("*").alias("Distinct_Organisation_Codes"))
    .withColumn("Submission_Month", F.lit(latest_month))
)

display(result_df)